In [1]:
from sites import sites
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By

конвертация месяц-номер

In [2]:
month_lib = {'января':'01','февраля':'02','марта':'03','апреля':'04','мая':'05','июня':'06','июля':'07','августа':'08','сентября':'09','октября':'10','ноября':'11','декабря':'12'}

### <a id='main'>Выполняемость сайтов</a> (меню, можно переходить по ссылкам на названиях)

<s>[УРГУПС (URUST)](#1) - 'https://www.usurt.ru/main-news'</s>  
<s>[СГУПС (STU)](#2) - 'http://www.stu.ru/news/'</s>  
<s>[УИ ГА (UVAUGA)](#3) - 'https://uvauga.ru'</s>  
[УГА (SPBGUGA)](#4) - 'https://spbguga.ru/category/news/' - code 500  
[МИИТ (MIIT)](#5) - 'https://www.miit.ru/news?from=1&to=50&category_id=1383,1384,1127,1363'  
[(SSUWT)](#6) -  
[РГУПС (RGUPS)](#7) - 'https://www.rgups.ru/news/'  
[ПГУПС (PGUPS)](#8) - 'https://pgups.ru/news/'  
[ОМГУПС (OMGUPS)](#9) - 'https://www.omgups.ru/news/'  
[ИГРУПС (IRGUPS)](#10) - 'https://www.irgups.ru/news'  
[ДВГУПС (DVGUPS)](#11) - 'https://dvgups.ru/press-main-page'  
[ГУМРФ (GUMRF)](#12) - 'https://gumrf.ru/news/'  
[САМГУПС (SAMGUPS)](#13) - 'https://www.samgups.ru/?PAGEN_1='  
[МГУ им. адм. Ф. Ф. Ушакова (AUMSU)](#14) - 'http://pk.aumsu.ru/novosti'  
[МГУ им. адм. Г.И. Невельского (MSUN)](#15) - 'https://www.msun.ru/ru/news/arch'

In [5]:
options = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/codespace/.cache/selenium/chrome/linux64/118.0.5993.70/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x56336c1f9fb3 <unknown>
#1 0x56336becd4a7 <unknown>
#2 0x56336bf00c93 <unknown>
#3 0x56336befd10c <unknown>
#4 0x56336bf3fac6 <unknown>
#5 0x56336bf36713 <unknown>
#6 0x56336bf0918b <unknown>
#7 0x56336bf09f7e <unknown>
#8 0x56336c1bf8d8 <unknown>
#9 0x56336c1c3800 <unknown>
#10 0x56336c1cdcfc <unknown>
#11 0x56336c1c4418 <unknown>
#12 0x56336c19142f <unknown>
#13 0x56336c1e84e8 <unknown>
#14 0x56336c1e86b4 <unknown>
#15 0x56336c1f9143 <unknown>
#16 0x7fe213922609 start_thread


## <a id = '1'>URUST</a>


[up to menu](#main)

In [ ]:
page = requests.get('https://www.usurt.ru/main-news')
print('_-'*70, '\n urust __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')


soup = bs(page.text, "html.parser") #варим суп по странице всех новостей
all_hrefs = soup.findAll('a', class_="news__button-more") #вытаскиваем все элементы с ссылками на отдельные новости

all_dates = soup.find_all('div', attrs={'style':'font-weight: 700; margin-bottom: 10px;'}) #даты
posts_usurt = {}
key = 'dt'
for date in all_dates:
    date = str(date.text)
    date = date[6:8]+'/'+date[3:5]+'/'+date[0:2]
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(date)

all_img_hrefs = soup.find_all('div',  class_='news-row__preview-img') #ссылки на картинки 
key = 'img_href'
for img_href in all_img_hrefs:
    img_href = img_href.get('style')
    img_href = img_href[23:-3]
    
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(img_href)

for link in all_hrefs: 
    href = link.get('href') #вытаскиваем ссылки из элементов поочередно
    news_link = f'https://www.usurt.ru{href}' #достраиваем ссылку до полной
    page_news = requests.get(news_link) #запрос на индивидуальную новость
    
    print(news_link, '__connection status__ -', page_news.status_code, '\n', '_'*50, '\n') #проверка связи


    news_soup = bs(page_news.text, "html.parser") #варим новый суп из странички с новостью

    key = 'name'
    name = news_soup.find_all('h1')[1] #вытаскиваем элемент заголовка
    # print("Заголовок\n", name.text, '\n', '-'*20)
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(name.text)

    key = 'text'
    box = news_soup.find_all('span', attrs={'style':'font-size:18px;'}) #вытягиваем все элементы с текстом
    text = '' #переменная для конечного результата
    for elem in box: #перебираем элементы поочередно и вытаскивает текст
        text+=elem.text
    # print("Основной текст\n", text, '\n', '-'*30, '\n\n')
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(text)   


In [ ]:
posts_usurt = pd.DataFrame.from_dict(posts_usurt)
posts_usurt

In [ ]:
posts_usurt.to_csv('posts/posts_usurt.csv')

## <a id='2'>STU</a>  

[up to menu](#main)

In [ ]:
posts_stu = {}

for i in range(59):    
    url = f'http://www.stu.ru/news/index.php?page=527&podrazd=&p={i}'
    
    page = requests.get(url)
    print('_-'*70, '\n urust __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')


    soup = bs(page.text, "html.parser") #варим суп по странице всех новостей

    
    hrefs = soup.find_all('a', class_=('newsLink'))
    all_img_hrefs = soup.find_all('img', class_=('no_zoom_plz'))

    key = 'img_href'
    for img_href in all_img_hrefs:
        img_href = img_href.get('src')
        img_href = f'http://www.stu.ru{img_href}'
        posts_stu.setdefault(key, [])
        posts_stu[key].append(img_href)


    for href in hrefs:
        link = href.get('href')
        link = f'http://www.stu.ru/news/{link}'

        link_page = requests.get(link)
        link_soup = bs(link_page.text, "html.parser")


        div = link_soup.find_all('div', {'id':'page_text'})


        key = 'dt'
        for date in div:
            date = date.find('p').text
            date = date[0:10]
            date = date[8:10]+'/'+date[3:5]+'/'+date[0:2]
            posts_stu.setdefault(key, [])
            posts_stu[key].append(date)


        key = 'name'
        for name in div:
            name = name.find('p').text
            name = name[11:-1]
            posts_stu.setdefault(key, [])
            posts_stu[key].append(name)

            

        key = 'text'
        text_div = link_soup.find_all('p', attrs={'style':'margin-top: 1em;'})
        for textus in text_div:
            textus = textus.text
            posts_stu.setdefault(key, [])
            posts_stu[key].append(textus)

    print('страница ',i,'\n')


In [ ]:
posts_stu = pd.DataFrame.from_dict(posts_stu)
posts_stu

In [ ]:
posts_stu.to_csv('posts/posts_stu.csv')

## <a id='3'>UVAUGA</a>

[up to menu](#main)

In [ ]:
page = requests.get('https://uvauga.ru')
print('_-'*70, '\n uvauga __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

soup = bs(page.text, "html.parser")
posts_uvauga = {}


#Сбор всех ссылок на новости (Основная страница + нижние "другие новости")

hrefs = soup.find_all("p", class_='readmore')
count = 1
all_hrefs = []

key = 'name'
for href in hrefs: #ссылки на индивидуальные новости
    name = href.text
    href = href.find("a").get("href")
    href = f'https://uvauga.ru/{href}'
    # print(f'{count}', href)
    # count += 1
    all_hrefs.append(href)
    posts_uvauga.setdefault(key, [])
    posts_uvauga[key].append(name)
    
    

hrefs_more = soup.find_all('div', class_='items-more')

for href_more in hrefs_more:
    href_more = href_more.find_all("a")

    for href_rez in href_more:
        name = href_rez.text
        href_rez = href_rez.get('href')
        href_rez = f'https://uvauga.ru{href_rez}'
        # print(f'{count}', href_rez)
        # count+=1
        all_hrefs.append(href_rez)
        posts_uvauga.setdefault(key, [])
        posts_uvauga[key].append(name)


In [ ]:
#Залезим в новости
for href in all_hrefs:
    page = requests.get(href)
    print('_-'*70, f'\n {href} __connection status__ -', page.status_code, '\n', '_-'*30, '\n\n')

    if page.status_code != 200:
        print(f'Ошибка подключения к странице {href}')
        break

    soup = bs(page.text, "html.parser")

    key = 'text'
    texts = soup.find_all('div', class_='item-page')
    all_imgs = texts
    for text in texts:
        text = text.text
        print(text,'_-'*20,'\n')
        posts_uvauga.setdefault(key, [])
        posts_uvauga[key].append(text)
    key = 'imgs'
    for imgs in all_imgs:
        imgs = imgs.find_all('img')
        img_rez = []
        if imgs != None:
            for img in imgs:
                img = img.get('src')
                img = f'https://uvauga.ru/{img}'
                img_rez.append(img)
            print(img_rez)

        posts_uvauga.setdefault(key, [])
        posts_uvauga[key].append(img_rez)



In [ ]:
print('_-'*50,'\n')
print(len(all_hrefs))
print(len(posts_uvauga['name']))
print(len(posts_uvauga['text']))
print(len(posts_uvauga['imgs']))
print(posts_uvauga['imgs'])

In [ ]:
posts_uvauga = pd.DataFrame.from_dict(posts_uvauga)
posts_uvauga

In [ ]:
posts_uvauga.to_csv('posts/posts_uvauga.csv')

## <a id='4'>SPBGUGA</a>

[up to menu](#main)

In [ ]:
url = 'https://spbguga.ru/category/news/'

page = requests.get(url)
print('_-'*70, '\n spbguga __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
print(page.status_code)

нету коннекта, не пущает, пропускаем

## <a id="5">MIIT</a>

[up to menu](#main)

100 страниц новостей  
update: До 2009 года - 530 страниц новостей

In [3]:
#0-530
for last in range(1,531):
    first = last - 1
    url = f'https://www.miit.ru/news?from={first}&to={last}' #посты с 1 по 50 страницу

    page = requests.get(url, timeout=1)
    print('_-'*70,f'\n {url} list {last} __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')
    
    soup = bs(page.text,"html.parser")
    print(soup)

    if last >= 1:
        break

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 https://www.miit.ru/news?from=0&to=1 list 1 __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 



<!DOCTYPE html>

<html lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no, maximum-scale=1, user-scalable=no" name="viewport"/>
<title>Новости | РУТ (МИИТ)</title>
<link href="/content/apple-icon-57x57.png?id_wm=803934" rel="apple-touch-icon-precomposed" sizes="57x57"/>
<link href="/content/apple-icon-60x60.png?id_wm=803936" rel="apple-touch-icon-precomposed" sizes="60x60"/>
<link href="/content/apple-icon-72x72.png?id_wm=803938" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/content/apple-icon-76x76.

## <a id='6'>SSUWT<a>

[up to menu](#main)

In [ ]:
url = "https://www.ssuwt.ru/?limit=12&limitstart=0"

page = requests.get(url)
print('_-'*70, '\n ssuwt __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")
hmp = soup.find('div', class_='k2PaginationCounter')
hmp = int(hmp.text[27:-8])

In [ ]:
posts_ssuwt = {}
for i in range(0,(hmp-1)*12,12):

    url = f"https://www.ssuwt.ru/?limit=12&limitstart={i}"
    page = requests.get(url)
    print('_-'*70, f'\n ssuwt_{int((i/12)+1)} __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

    soup = bs(page.text, "html.parser")


    key = 'dt'
    all_dt = soup.find_all('span', class_=('catItemDateCreated'))
    for date in all_dt:
        date = date.text[13:-8].split(',')[1]

        day = date.split()[0]
        month = month_lib[date.split()[1]]
        year = date.split()[2]

        time = date.split()[3]

        date = day+'.'+month+'.'+year+' '+time

        posts_ssuwt.setdefault(key, [])
        posts_ssuwt[key].append(date)


    h3 = soup.find_all('h3', class_=('catItemTitle'))

    
    key = 'name'
    for post in h3:
        elem = post.find('a')

        name = elem.text[17:-12]
        posts_ssuwt.setdefault(key, [])
        posts_ssuwt[key].append(name)


    key = 'text'
    for post in h3:
        elem = post.find('a')

        href = elem.get('href')
        link = f'https://www.ssuwt.ru/{href}'

        link_page = requests.get(link)
        soup = bs(link_page.text, "html.parser")

        textus = soup.find_all('div', class_=('itemFullText'))
        for text in textus:
            text = text.text

            posts_ssuwt.setdefault(key, [])
            posts_ssuwt[key].append(text)

    

    print(posts_ssuwt)


In [ ]:
posts_ssuwt = pd.DataFrame.from_dict(posts_ssuwt)
posts_ssuwt

## <a id="7">RGUPS</a>

[up to menu](#main)

In [ ]:
url = 'http://www.rgups.ru/news/page1'

page = requests.get(url)
print('_-'*70, '\n rgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, 'html.parser')

## <a id="8">PGUPS</a>

[up to menu](#main)

In [ ]:
url = 'https://pgups.ru/news/?PAGEN_2=1'

page = requests.get(url)
print('_-'*70, '\n pgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")

## <a id='9'>OMGUPS</a>

[up to menu](#main)

In [ ]:
soup = bs(page.text, "html.parser")

## <a id='10'>IRGUPS</a>

[up to menu](#main)

In [ ]:
url = 'https://www.irgups.ru/news?page=1'

page = requests.get(url)
print('_-'*70, '\n irgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")

## <a id='11'>DVGUPS</a>

[up to menu](#main)

In [ ]:
url = 'https://dvgups.ru/press-main-page?start=0'

page = requests.get(url)
print('_-'*70, '\n dvgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")

## <a id="12">GUMRF</a>

[up to menu](#main)

In [ ]:
url = 'https://gumrf.ru/news2023/2'

page = requests.get(url)
print('_-'*70, '\n gumrf __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")

## <a id=13>SAMGUPS</a>

[up to menu](#main)

In [ ]:
url = 'https://www.samgups.ru/?PAGEN_1=1'

page = requests.get(url)
print('_-'*70, '\n samgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")

## <a id='14'>AUMSU</a>

[up to menu](#main)

In [ ]:
url = 'http://pk.aumsu.ru/novosti'

page = requests.get(url)
print('_-'*70, '\n aumsu __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")

## <a id='15'>MSUN</a>

[up to menu](#main)

In [ ]:
url = 'https://www.msun.ru/ru/news/arch'

page = requests.get(url)
print('_-'*70, '\n msun __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

In [ ]:
soup = bs(page.text, "html.parser")